PS: Had made changes to LDA2Vec. Did not work.
TF-IDF with LSTM, XGBoost not working. Didn't do for SVM. If I did, then didn't check if it was working.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# GLOVE_VECTORS_FILE = "/content/gdrive/MyDrive/NLP_Project/glove.twitter.27B.50d.txt"
GLOVE_VECTORS_FILE = "/content/gdrive/MyDrive/NLP_Project/glove.twitter.27B.100d.txt"
TRAIN_FILE = "/content/gdrive/MyDrive/NLP_Project/train.csv"

In [ ]:
# Using embeddings for MLP
import sklearn
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import load_model
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import pickle
import re

import numpy as np

import pandas as pd
import string
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=3)

In [ ]:
df = pd.read_csv(TRAIN_FILE)
# df_test = pd.read_csv(TEST_FILE)

In [ ]:
#dropping columns that will not be used
df = df.drop(columns = ['id','keyword','location'])

In [ ]:
print(df.columns)

Index(['text', 'target'], dtype='object')


### **PRE PROCESSING THE TRAINING DATA**

In [ ]:
df_clean = df
#converting to lower case
df_clean['text'] = df.text.str.lower()
#removing duplicate tweets
df_clean.drop_duplicates(subset='text',inplace=True)
# for internet links starting with http/s and hashtags and user mentions using regular exp
df_clean.text = df_clean.text.apply(lambda x: re.sub(r'(?:\@|\#|https?:\/\/)\S+', '', x))
 # for intenet links starting with www
df_clean.text = df_clean.text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
#removing digits
df_clean.text = df_clean.text.apply(lambda x: re.sub(r'[0-9]', '', x))
#removing punctuations
df_clean.text = df_clean.text.apply(lambda x: re.sub(r'[\(\)\-\:\;\!\.\,\?\\\/\*\&\^\%\$\#\@\"\<\>\+\=\-\_\[\]\{\}\~\`\']', ' ', x))

In [ ]:
# tokenizing
df_clean['tokens'] = df_clean['text'].apply(tknzr.tokenize)

In [ ]:
df_clean.columns

Index(['text', 'target', 'tokens'], dtype='object')

In [ ]:
#shuffling the rows in the data frame
df_clean = df_clean.sample(frac = 1)

## **I) WORD2VEC**

In [ ]:
# import gensim, import word2vec, fit on your tokens 

In [ ]:
# from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model_word2vec = Word2Vec(sentences=df_clean['tokens'], size=100, window=5, min_count=1, workers=4)
model_word2vec.save("word2vec.model")

In [ ]:
#model['razed']

In [ ]:
max_features = 30000 # total number of words in vocabulary 
max_len = 100  # maximum input token length
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df_clean['text'])

X = tokenizer.texts_to_sequences(df_clean['text'])
X = pad_sequences(X, maxlen = 100)
Y = pd.get_dummies(df_clean['target']).values

In [ ]:
v_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((v_size, 100))
for word, index in tokenizer.word_index.items():
  if word not in model_word2vec.wv.vocab:
    embedding_vector = np.zeros(100)
  else:
    embedding_vector = model_word2vec[word]
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, stratify=Y)

#I)A) LDA2Vec

In [ ]:
# from gensim.test.utils import common_texts
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary

common_text = df_clean['tokens']

# Create a corpus from a list of texts
common_dictionary = Dictionary(df_clean['tokens'])
common_corpus = [common_dictionary.doc2bow(text) for text in common_text]

# Train the model on the corpus.
num_topics = 15
lda = LdaModel(common_corpus, num_topics=num_topics)

# print(common_text)

In [ ]:
def LDA_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model_word2vec.show_topic(i, topn = 10);
        word_dict['Words of Topic ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [ ]:
LDA_topics(lda, num_topics)

In [ ]:
from gensim.models import CoherenceModel
coherence_using_lda2vec = CoherenceModel(model=lda, texts=common_text, dictionary=common_dictionary, coherence='c_v')
coherence = coherence_using_lda2vec.get_coherence()
print('Coherence Score: ', round(coherence, 3))

Coherence Score:  0.302


In [ ]:
svd_matrix = lda.fit(common_corpus)

In [ ]:
print(type(coherence_using_lda2vec))

<class 'gensim.models.coherencemodel.CoherenceModel'>


In [ ]:
X_lr = 
Y_lr = df_clean['target']
X_lr_train, X_lr_test, Y_lr_train, Y_lr_test = train_test_split(X_lr, Y_lr, test_size = 0.2, random_state = 1)
clf = LogisticRegression(random_state=0).fit(X_lr_train, Y_lr_train)

# I)B) MLP WITH WORD2VEC EMBEDDINGS

In [ ]:
maxlen = 100
model_word2vec_mlp = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(500,activation='relu'),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_word2vec_mlp.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_word2vec_mlp.summary()

In [ ]:
model_word2vec_mlp.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1,  callbacks=[early_stopping])

In [ ]:
model_word2vec_mlp.evaluate(X_test, Y_test)

In [ ]:
y_pred = model_word2vec_mlp.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

# I)C) LSTM with Word2Vec embeddings

In [ ]:
maxlen = 100
model_word2vec_lstm = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.LSTM(128),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_word2vec_lstm.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_word2vec_lstm.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 100, 100)          1324300   
                                                                 
 lstm_13 (LSTM)              (None, 128)               117248    
                                                                 
 dense_19 (Dense)            (None, 2)                 258       
                                                                 
Total params: 1,441,806
Trainable params: 1,441,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_word2vec_lstm.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/3
169/169 [==============================] - 28s 150ms/step - loss: 0.6516 - accuracy: 0.6080 - val_loss: 0.5112 - val_accuracy: 0.7687
Epoch 2/3
169/169 [==============================] - 25s 150ms/step - loss: 0.3872 - accuracy: 0.8374 - val_loss: 0.4563 - val_accuracy: 0.7953


In [ ]:
model_word2vec_lstm.evaluate(X_test, Y_test)

47/47 [==============================] - 2s 37ms/step - loss: 0.4515 - accuracy: 0.7948


[0.45150139927864075, 0.7948034405708313]

In [ ]:
y_pred = model_word2vec_lstm.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.82      0.82      0.82       861
           1       0.76      0.77      0.76       640

    accuracy                           0.79      1501
   macro avg       0.79      0.79      0.79      1501
weighted avg       0.80      0.79      0.79      1501



##I)D) XGBOOST with WORD2VEC Embeddings

In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    temp += np.array(model_word2vec[df_clean.iloc[i]['tokens'][j]])
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
xgb_input = np.array(xgb_input)
print(xgb_input.shape)

(7502, 100)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)

In [ ]:
cross_val_score(XGBClassifier(), xgb_input, df_clean['target'])

array([0.6835443 , 0.69686875, 0.67666667, 0.68      , 0.69066667])

In [ ]:
cross_val_score(XGBClassifier(), xgb_input, df_clean['target']).mean()

0.6855492782589385

##I)E) SVM with WORD2VEC Embeddings

In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    temp += np.array(model_word2vec[df_clean.iloc[i]['tokens'][j]])
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

xgb_input = np.array(xgb_input)

X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(sklearn.metrics.accuracy_score(Y_test, y_pred))

0.6835443037974683


In [ ]:
print(sklearn.metrics.classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.82      0.75       861
           1       0.67      0.50      0.57       640

    accuracy                           0.68      1501
   macro avg       0.68      0.66      0.66      1501
weighted avg       0.68      0.68      0.67      1501



## **II) FastText**

In [ ]:
from gensim.models import FastText

model_fast = FastText(size=100, window=3, min_count=1, sentences=df_clean['tokens'], workers=10)
model_fast.save("fasttext.model")

In [ ]:
# model_fast.wv.most_similar("accident")

In [ ]:
max_features = 30000 # total number of words in vocabulary 
max_len = 100  # maximum input token length
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df_clean['text'])

X = tokenizer.texts_to_sequences(df_clean['text'])
X = pad_sequences(X, maxlen = 100)
Y = pd.get_dummies(df_clean['target']).values

In [ ]:
v_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((v_size, 100))
for word, index in tokenizer.word_index.items():
  if word not in model_fast.wv.vocab:
    embedding_vector = np.zeros(100)
  else:
    embedding_vector = model_fast[word]
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, stratify=Y)

## II)A) MLP with FastText embeddings

In [ ]:
maxlen = 100
model_fast_mlp = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(500,activation='relu'),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_fast_mlp.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_fast_mlp.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, 100, 100)          1324300   
                                                                 
 flatten_10 (Flatten)        (None, 10000)             0         
                                                                 
 dense_33 (Dense)            (None, 500)               5000500   
                                                                 
 dense_34 (Dense)            (None, 2)                 1002      
                                                                 
Total params: 6,325,802
Trainable params: 6,325,802
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_fast_mlp.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/3
169/169 [==============================] - 12s 69ms/step - loss: 0.6987 - accuracy: 0.5881 - val_loss: 0.6400 - val_accuracy: 0.5957
Epoch 2/3
169/169 [==============================] - 12s 68ms/step - loss: 0.4834 - accuracy: 0.7728 - val_loss: 0.5815 - val_accuracy: 0.6789


In [ ]:
model_fast_mlp.evaluate(X_test, Y_test)

47/47 [==============================] - 1s 12ms/step - loss: 0.5530 - accuracy: 0.7109


[0.5529842376708984, 0.7108594179153442]

In [ ]:
y_pred = model_fast_mlp.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.83      0.62      0.71       861
           1       0.62      0.83      0.71       640

    accuracy                           0.71      1501
   macro avg       0.73      0.73      0.71      1501
weighted avg       0.74      0.71      0.71      1501



## II)B) LSTM with FastText embeddings

In [ ]:
maxlen = 100
model_fast_lstm = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.LSTM(128),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_fast_lstm.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_fast_lstm.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_20 (Embedding)    (None, 100, 100)          1324300   
                                                                 
 lstm_19 (LSTM)              (None, 128)               117248    
                                                                 
 dense_35 (Dense)            (None, 2)                 258       
                                                                 
Total params: 1,441,806
Trainable params: 1,441,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_fast_lstm.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/3
169/169 [==============================] - 27s 148ms/step - loss: 0.6481 - accuracy: 0.6117 - val_loss: 0.5153 - val_accuracy: 0.7587
Epoch 2/3
169/169 [==============================] - 25s 146ms/step - loss: 0.4330 - accuracy: 0.8144 - val_loss: 0.4993 - val_accuracy: 0.7554


In [ ]:
model_fast_lstm.evaluate(X_test, Y_test)

47/47 [==============================] - 2s 35ms/step - loss: 0.4817 - accuracy: 0.7821


[0.48170867562294006, 0.7821452617645264]

In [ ]:
y_pred = model_fast_lstm.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       861
           1       0.73      0.78      0.75       640

    accuracy                           0.78      1501
   macro avg       0.78      0.78      0.78      1501
weighted avg       0.78      0.78      0.78      1501



In [ ]:
# y_pred[:3]

In [ ]:
sklearn.metrics.accuracy_score(y_test_single, y_pred_single)

0.782145236508994

##II)C) XGBOOST with FastText Embeddings

In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    temp += np.array(model_fast[df_clean.iloc[i]['tokens'][j]])
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

xgb_input = np.array(xgb_input)
# print(xgb_input.shape)

X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
cross_val_score(XGBClassifier(), xgb_input, df_clean['target'])

array([0.70486342, 0.67554963, 0.68      , 0.67933333, 0.69066667])

##II)D) SVM with FastText Embeddings


In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    temp += np.array(model_fast[df_clean.iloc[i]['tokens'][j]])
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

xgb_input = np.array(xgb_input)
# print(xgb_input.shape)

X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)

In [ ]:
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)

y_pred = clf.predict(X_test)

In [ ]:
print(sklearn.metrics.accuracy_score(Y_test, y_pred))

0.6915389740173218


In [ ]:
print(sklearn.metrics.classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.69      0.83      0.76       861
           1       0.69      0.50      0.58       640

    accuracy                           0.69      1501
   macro avg       0.69      0.67      0.67      1501
weighted avg       0.69      0.69      0.68      1501



## **III) GLoVE**

In [ ]:
word_to_vec_map = {}
with open(GLOVE_VECTORS_FILE, 'r', encoding='UTF-8') as f:
    words = set()
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

In [ ]:
# print(word_to_vec_map['the'])

In [ ]:
max_features = 30000 # total number of words in vocabulary 
max_len = 100  # maximum input token length
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df_clean['text'])

X = tokenizer.texts_to_sequences(df_clean['text'])
X = pad_sequences(X, maxlen = 100)
Y = pd.get_dummies(df_clean['target']).values

In [ ]:
v_size = len(tokenizer.word_index) + 1
embed_vector_len = word_to_vec_map['the'].shape[0]
embedding_matrix = np.zeros((v_size, 100))

for word, index in tokenizer.word_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    embedding_matrix[index, :] = embedding_vector

In [ ]:
# word_to_vec_map.get('the').shape

(100,)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, stratify=Y)

## III)A) MLP WITH GLOVE WORD EMBEDDINGS

In [ ]:
maxlen = 100
model_glove_mlp = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(500,activation='relu'),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_glove_mlp.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_glove_mlp.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 100, 100)          1312600   
                                                                 
 flatten_12 (Flatten)        (None, 10000)             0         
                                                                 
 dense_51 (Dense)            (None, 500)               5000500   
                                                                 
 dense_52 (Dense)            (None, 2)                 1002      
                                                                 
Total params: 6,314,102
Trainable params: 6,314,102
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_glove_mlp.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/3
169/169 [==============================] - 12s 69ms/step - loss: 0.4935 - accuracy: 0.7674 - val_loss: 0.4339 - val_accuracy: 0.8203
Epoch 2/3
169/169 [==============================] - 12s 69ms/step - loss: 0.3130 - accuracy: 0.8731 - val_loss: 0.4725 - val_accuracy: 0.8186
Epoch 3/3
169/169 [==============================] - 12s 68ms/step - loss: 0.1746 - accuracy: 0.9359 - val_loss: 0.6191 - val_accuracy: 0.8003


In [ ]:
model_glove_mlp.evaluate(X_test, Y_test)

47/47 [==============================] - 1s 12ms/step - loss: 0.5988 - accuracy: 0.7935


[0.5988075137138367, 0.7934710383415222]

In [ ]:
y_pred = model_glove_mlp.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83       861
           1       0.80      0.69      0.74       640

    accuracy                           0.79      1501
   macro avg       0.79      0.78      0.78      1501
weighted avg       0.79      0.79      0.79      1501



## III)B) LSTM WITH GLOVE WORD EMBEDDINGS

In [ ]:
maxlen = 100
model_glove_lstm = tf.keras.Sequential([
         tf.keras.layers.Embedding(v_size, 100, weights=[embedding_matrix],input_length = maxlen),
         tf.keras.layers.LSTM(128),
         tf.keras.layers.Dense(2,activation='softmax')                  
        ])
model_glove_lstm.compile(loss = "binary_crossentropy",optimizer = 'adam',metrics = ['accuracy'])
model_glove_lstm.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 100, 100)          1324300   
                                                                 
 lstm_20 (LSTM)              (None, 128)               117248    
                                                                 
 dense_38 (Dense)            (None, 2)                 258       
                                                                 
Total params: 1,441,806
Trainable params: 1,441,806
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_glove_lstm.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/3
169/169 [==============================] - 25s 150ms/step - loss: 0.0527 - accuracy: 0.9796 - val_loss: 1.0852 - val_accuracy: 0.7504
Epoch 2/3
169/169 [==============================] - 25s 148ms/step - loss: 0.0443 - accuracy: 0.9802 - val_loss: 1.2219 - val_accuracy: 0.7488


In [ ]:
model_glove_lstm.evaluate(X_test, Y_test)

47/47 [==============================] - 2s 36ms/step - loss: 1.0793 - accuracy: 0.7648


[1.0792698860168457, 0.7648234367370605]

In [ ]:
y_pred = model_glove_lstm.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.77      0.83      0.80       861
           1       0.75      0.67      0.71       640

    accuracy                           0.76      1501
   macro avg       0.76      0.75      0.76      1501
weighted avg       0.76      0.76      0.76      1501



##III)C) XGBOOST with GLOVE Embeddings

In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    embedding_vector = word_to_vec_map.get(df_clean.iloc[i]['tokens'][j])
    if embedding_vector is not None:
      temp += embedding_vector
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

In [ ]:
xgb_input = np.array(xgb_input)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)


In [ ]:
cross_val_score(XGBClassifier(), xgb_input, df_clean['target'])

array([0.80946036, 0.80546302, 0.78533333, 0.798     , 0.79666667])

##III)E) SVM with GLoVE Embeddings

In [ ]:
xgb_input = []
for i in range(len(df_clean['tokens'])):
  temp = np.zeros(100, dtype=float)
  for j in range(len(df_clean.iloc[i]['tokens'])):
    embedding_vector = word_to_vec_map.get(df_clean.iloc[i]['tokens'][j])
    if embedding_vector is not None:
      temp += embedding_vector
  temp /= len(df_clean.iloc[i]['tokens'])
  xgb_input.append(temp)

xgb_input = np.array(xgb_input)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean['target'], stratify=df_clean['target'],test_size=0.2, random_state=0)

In [ ]:
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, Y_train)

y_pred = clf.predict(X_test)

In [ ]:
print(sklearn.metrics.accuracy_score(Y_test, y_pred))

0.8094603597601598


In [ ]:
print(sklearn.metrics.classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       861
           1       0.84      0.68      0.75       640

    accuracy                           0.81      1501
   macro avg       0.82      0.79      0.80      1501
weighted avg       0.81      0.81      0.81      1501



##**IV) TF-IDF**

In [ ]:
df_clean_tfidf = df_clean

In [ ]:
# removing stop words

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

df_clean_tfidf['text'] = df_clean_tfidf['text'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

v = TfidfVectorizer()
x = v.fit_transform(df_clean_tfidf['text'])
x = x.toarray()

Y = pd.get_dummies(df_clean['target']).values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, Y, test_size = 0.1, random_state = 1)

In [ ]:
vocab = v.get_feature_names_out()
print(vocab[-1])
print(vocab.shape)

ûówe
(12870,)


In [ ]:
# print(X[10])
c = 0
for i in x:
  for j in i:
    if j == 1:
      c += 1

In [ ]:
print(c)

115


## IV)A) MLP WITH TF-IDF VECTORS

In [ ]:
maxlen = 100

model_tfidf_mlp = tf.keras.Sequential()
# model_tfidf_mlp.add(tf.keras.layers.Flatten())
model_tfidf_mlp.add(tf.keras.layers.Dense(500, activation='relu'))
model_tfidf_mlp.add(tf.keras.layers.Dense(2, activation='softmax'))
model_tfidf_mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_tfidf_mlp.build(X_train.shape)
model_tfidf_mlp.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (6751, 500)               6435500   
                                                                 
 dense_9 (Dense)             (6751, 2)                 1002      
                                                                 
Total params: 6,436,502
Trainable params: 6,436,502
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_tfidf_mlp.fit(X_train, Y_train, epochs = 5, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
190/190 [==============================] - 9s 47ms/step - loss: 0.5466 - accuracy: 0.7213 - val_loss: 0.4286 - val_accuracy: 0.8166
Epoch 2/5
190/190 [==============================] - 9s 45ms/step - loss: 0.2818 - accuracy: 0.8871 - val_loss: 0.4515 - val_accuracy: 0.8062
Epoch 3/5
190/190 [==============================] - 9s 45ms/step - loss: 0.1474 - accuracy: 0.9505 - val_loss: 0.5067 - val_accuracy: 0.7914
Epoch 4/5
190/190 [==============================] - 9s 45ms/step - loss: 0.0927 - accuracy: 0.9691 - val_loss: 0.5883 - val_accuracy: 0.7840


In [ ]:
model_tfidf_mlp.evaluate(X_test, Y_test)

24/24 [==============================] - 0s 15ms/step - loss: 0.6846 - accuracy: 0.7577


[0.6846101880073547, 0.757656455039978]

In [ ]:
y_pred = model_tfidf_mlp.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

              precision    recall  f1-score   support

           0       0.79      0.81      0.80       435
           1       0.73      0.70      0.71       316

    accuracy                           0.76       751
   macro avg       0.76      0.75      0.75       751
weighted avg       0.76      0.76      0.76       751



## IV)B) LSTM WITH TF-IDF VECTORS

In [ ]:
X_train_lstm = X_train[:, :, None]
print(X_train_lstm.shape)

(6751, 12870, 1)


In [ ]:
maxlen = 100

model_tfidf_lstm = tf.keras.Sequential()
# model_tfidf_lstm.add(tf.keras.layers.LSTM(100, input_shape=(None, data_dim),return_sequences=True))
model_tfidf_lstm.add(tf.keras.layers.LSTM(units=6, input_shape = X_train_lstm.shape, return_sequences = True))
model_tfidf_lstm.add(tf.keras.layers.LSTM(128))
model_tfidf_lstm.add(tf.keras.layers.Dense(2,activation='softmax'))
model_tfidf_lstm.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])
model_tfidf_lstm.summary()

# model = Sequential()
# model.add(LSTM(100, input_shape=(None, data_dim),return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(200))
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model_tfidf_lstm.fit(X_train, Y_train, epochs = 3, verbose = 1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
model_tfidf_lstm.evaluate(X_test, Y_test)

47/47 [==============================] - 4s 47ms/step - loss: 0.6842 - accuracy: 0.5676


[0.6841512322425842, 0.5676215887069702]

In [ ]:
y_pred = model_tfidf_lstm.predict(X_test)
y_pred_single = [np.argmax(i) for i in y_pred]
y_test_single = [np.argmax(i) for i in Y_test]
print(sklearn.metrics.classification_report(y_test_single, y_pred_single))

## IV)C) XGBoost with TF-IDF Vectors

In [ ]:
# xgb_input = []
# for i in range(len(df_clean['tokens'])):
#   temp = np.zeros(100, dtype=float)
#   for j in range(len(df_clean.iloc[i]['tokens'])):
#     temp += np.array(model_fast[df_clean.iloc[i]['tokens'][j]])
#   temp /= len(df_clean.iloc[i]['tokens'])
#   xgb_input.append(temp)

xgb_input = x
xgb_input = np.array(xgb_input)

X_train, X_test, Y_train, Y_test = train_test_split(xgb_input, df_clean_tfidf['target'], stratify=df_clean_tfidf['target'],test_size=0.2, random_state=0)

In [ ]:
print(xgb_input)

In [ ]:
cross_val_score(XGBClassifier(), xgb_input, df_clean_tfidf['target'])

##**V) LSA**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
documents = df_clean['text']
  
# raw documents to tf-idf matrix: 
v = TfidfVectorizer()
# x = v.fit_transform(df_clean_tfidf['text'])

# SVD to reduce dimensionality: 
svd_model = TruncatedSVD(n_components=2,
                         algorithm='randomized',
                         n_iter=10)
# pipeline of tf-idf + SVD, fit to and applied to documents:
svd_transformer = Pipeline([('tfidf', v), 
                            ('svd', svd_model)])
svd_matrix = svd_transformer.fit_transform(documents)

print(svd_matrix.shape)
# svd_matrix can later be used to compare documents, compare words, or compare queries with documents

(7502, 2)


In [ ]:
topic_encoded_df = pd.DataFrame(svd_matrix, columns=["Topic1", "Topic2"])
# topic_encoded_df["text"] = df_clean['text']

# print(topic_encoded_df[["text", "Topic1", "Topic2"]][:5])

print(topic_encoded_df.shape)

(7502, 2)


In [ ]:
# dictionary = vectorizer.get_feature_names()
# print(dictionary)

In [ ]:
# encoding_matrix = pd.DataFrame(svd_model.components_, index=['Topic1', 'Topic2'], columns=dictionary).T
# print(encoding_matrix)

In [ ]:
# encoding_matrix['Topic1'] = np.abs(encoding_matrix['Topic1']) 
# encoding_matrix['Topic2'] = np.abs(encoding_matrix['Topic2'])

# encoding_matrix.sort_values('Topic1', ascending=True)
# encoding_matrix.sort_values('Topic2', ascending=True)

# print(encoding_matrix)


In [ ]:
X_lr = svd_matrix
Y_lr = df_clean['target']
X_lr_train, X_lr_test, Y_lr_train, Y_lr_test = train_test_split(X_lr, Y_lr, test_size = 0.2, random_state = 1)
clf = LogisticRegression(random_state=0).fit(X_lr_train, Y_lr_train)

clf.predict(X_lr)

array([1, 0, 0, ..., 0, 0, 0])

In [ ]:
y_pred = clf.predict(X_lr_test)
print(sklearn.metrics.classification_report(Y_lr_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.84      0.73       853
           1       0.65      0.38      0.48       648

    accuracy                           0.64      1501
   macro avg       0.65      0.61      0.60      1501
weighted avg       0.64      0.64      0.62      1501



##**VII) LDA**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

v = TfidfVectorizer()
x = v.fit_transform(df_clean_tfidf['text'])

lda_model = LatentDirichletAllocation(n_components = 2, max_iter = 20, random_state = 20)
# fit transform on model on our count vectorizer : running this will return our topics
X_topics = lda_model.fit_transform(x)

topic_words = lda_model.components_

print(topic_words)
print(len(X_topics))

[[1.04075982 0.84946913 0.895772   ... 0.96189213 0.51459884 0.94471675]
 [1.21622101 0.51704217 0.52082213 ... 0.51298004 0.88051812 0.51176519]]
7502


In [ ]:
X_topics

array([[0.44899263, 0.55100737],
       [0.8390509 , 0.1609491 ],
       [0.10909605, 0.89090395],
       ...,
       [0.85717554, 0.14282446],
       [0.81600903, 0.18399097],
       [0.15137731, 0.84862269]])

In [ ]:
X_lr = X_topics
Y_lr = df_clean_tfidf['target']
X_lr_train, X_lr_test, Y_lr_train, Y_lr_test = train_test_split(X_lr, Y_lr, test_size = 0.2, random_state = 1)
clf = LogisticRegression(random_state=1).fit(X_lr_train, Y_lr_train)

clf.predict(X_lr)

array([0, 0, 1, ..., 0, 0, 1])

In [ ]:
clf.score(X_lr_test, Y_lr_test)

0.5929380413057961

In [ ]:
y_pred = clf.predict(X_lr_test)
print(sklearn.metrics.classification_report(Y_lr_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.86      0.70       816
           1       0.62      0.27      0.38       685

    accuracy                           0.59      1501
   macro avg       0.60      0.57      0.54      1501
weighted avg       0.60      0.59      0.55      1501

